**Libraries**

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import sempy.fabric as fabric

**Define a logging dataframe**

In [ ]:
dfLogging = pd.DataFrame(columns = ['LoadId','NotebookId', 'NotebookName', 'WorkspaceId', 'CellId', 'Timestamp', 'ElapsedTime', 'Message', 'ErrorMessage'])
vContext = mssparkutils.runtime.context
vNotebookId = vContext["currentNotebookId"]
vLogNotebookName = vContext["currentNotebookName"]
vWorkspaceId = vContext["currentWorkspaceId"] # where the notebook is running, to not confuse with source and target workspaces

**Parameters --> convert to code for debugging the notebook. otherwise, keep commented as parameters are passed from DevOps pipelines**

pToken = ""
pSqlToken = ""
pSourceWorkspaceId = ""
pTargetWorkspaceId = ""
pDebugMode = "yes"
pFeatureBranch = ""
pOnelakeRoles = ''
pOnelakeRules = ''
pOnelakeEntraMembers = ''
pOnelakeItemMembers = ''

**Access token**

In [ ]:
vScope = "https://analysis.windows.net/powerbi/api"

# get the access token 
if pDebugMode == "yes":
    # in debug mode, use the token of the current user
    vAccessToken  = mssparkutils.credentials.getToken(vScope)
    vSqlAccessToken = vAccessToken
else:
    # when the code is run from the pipelines, to token is generated in a previous step and passed as a parameter to the notebook
    vAccessToken = pToken 
    vSqlAccessToken = pSqlToken

**Check if the source workspace passed from DevOps equals the feature branch name**
- This is a specific handling when a PR is done from the feature branch

In [ ]:
if pSourceWorkspaceId == pFeatureBranch:
    pSourceWorkspaceId = fabric.resolve_workspace_id(workspace=pFeatureBranch)

**Define the DAG**

In [ ]:
dagList = []

# add to the DAG list nb_cicd_pre_update_lakehouses
dagList.append({
            "name": "nb_cicd_pre_update_lakehouses",
            "path": "nb_cicd_pre_update_lakehouses",
            "timeoutPerCellInSeconds": 300,
            "args": {
                "useRootDefaultLakehouse": True,
                "pToken": vAccessToken,
                "pSqlToken": vSqlAccessToken,
                "pSourceWorkspaceId":pSourceWorkspaceId,
                "pTargetWorkspaceId":pTargetWorkspaceId,
                "pDebugMode":pDebugMode,
                "pOnelakeRoles":pOnelakeRoles,
                "pOnelakeRules":pOnelakeRules,
                "pOnelakeEntraMembers":pOnelakeEntraMembers,
                "pOnelakeItemMembers":pOnelakeItemMembers,
                }
        })

# add to the DAG list nb_cicd_pre_update_warehouses
dagList.append({
            "name": "nb_cicd_pre_update_warehouses",
            "path": "nb_cicd_pre_update_warehouses",
            "timeoutPerCellInSeconds": 300,
            "args": {
                "useRootDefaultLakehouse": True,
                "pSqlToken": vSqlAccessToken,
                "pSourceWorkspaceId":pSourceWorkspaceId,
                "pTargetWorkspaceId":pTargetWorkspaceId,
                "pDebugMode":pDebugMode
                },
            "dependencies": ["nb_cicd_pre_update_lakehouses"]
        })

DAG = { "activities": dagList,"concurrency": 2, "timeoutInSeconds": 900 }


**Run multiple**

In [ ]:
try:
    mssparkutils.notebook.runMultiple(DAG, {"displayDAGViaGraphviz": True})

    # logging
    vMessage = f"succeeded"
    dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, 'running the DAG', datetime.now(), None, vMessage, ''] 

except Exception as e:
    vMessage = f"failed"
    dfLogging.loc[len(dfLogging.index)] = [None, vNotebookId, vLogNotebookName, vWorkspaceId, 'running the DAG', datetime.now(), None, vMessage, str(e)]
    if pDebugMode == "yes":
        print(str(e))

**Logging**

In [ ]:
try:
    # perform the conversion of columns
    dfLogging = dfLogging.astype({
            "LoadId": "string",	
            "NotebookId": "string", 	
            "NotebookName": "string", 
            "WorkspaceId": "string", 
            "CellId": "string", 
            "Timestamp": "datetime64[ns]", 
            "ElapsedTime": "string", 
            "Message": "string", 
            "ErrorMessage" : "string"
        })

    # save panda dataframe to a spark dataframe 
    sparkDF_Logging = spark.createDataFrame(dfLogging) 

    # save to the lakehouse
    sparkDF_Logging.write.mode("append").format("delta").option("mergeSchema", "true").saveAsTable("staging.notebook_logging")

except Exception as e:
    vMessage = "saving logs to the lakehouse failed"
    if pDebugMode == "yes":
        print(str(e))

**Exit notebook**

In [ ]:
mssparkutils.notebook.exit(f"Notebook <{vLogNotebookName}> run successfully. Check logging table in CI/CD lakehouse for more details.")